In [115]:
import pandas as pd, numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [112]:
# NB: this requires openpyxl to be installed
xl = pd.ExcelFile("../data/Dataset.xlsx")
# extract sheets that start with consumer
consumer_sheets = [name for name in xl.sheet_names
                   if name.startswith("Consumer")]
# TODO: take all consumer sheets, not only the first 10 (= delete next line)
consumer_sheets = consumer_sheets[:20]

In [113]:
# mapping consumer to DataFrame containing tabular data
cons2df = {name: xl.parse(name).drop(columns=["Unnamed: 0","Total Consumption"])
           for name in tqdm(consumer_sheets)}

100%|██████████| 20/20 [01:29<00:00,  4.48s/it]


### Creation of dataset for task 1

In [116]:
# sum over all columns that are not Periods (which is at index 0)
sum_cols = cons2df["Consumer1"].columns[1:].tolist()
for name, df in cons2df.items():
    df["Total Consumption"] = df[sum_cols].sum(axis=1)

In [117]:
# constants based on 15min intervals
day_length = 4 * 24
week_window = day_length * 7

# iterate over DataFrames and extract x = week -> y = next day pairs
xs, ys = [], []
for df in cons2df.values():
  # extract the total consumption data from the dataset
  data = df["Total Consumption"].values

  # obtain start and end values of the window
  # NB: the window end includes the day to be predicted
  starts = np.arange(0, len(data) - week_window - day_length + 1, day_length)
  ends = starts + week_window + day_length # end includes the day to predict
  for start, end in zip(starts, ends):
    # extract the 8-day window from the dataset
    window = data[start:end]
    # split the window in week data and the day to predict
    x, y = window[:-day_length], window[-day_length:]
    # put the samples in the dataset
    xs.append(x)
    ys.append(y)
  assert end == len(data) # ensure that we covered all of the input data

# create length dimension
xs, ys = np.stack(xs), np.stack(ys)
xs.shape, ys.shape

((7180, 672), (7180, 96))

In [118]:
# generate train/test splits and save data
xtr, xte, ytr, yte = train_test_split(xs, ys, test_size=.1, random_state=42)
np.savez("../data/task1_train", x=xtr, y=ytr)
np.savez("../data/task1_test", x=xte, y=yte)